In [2]:
import nltk
import pandas as pd
df = pd.read_csv("metadata.csv")

C:\Users\18223\AppData\Local\Temp\ipykernel_16240\129582515.py:3: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("metadata.csv")


In [33]:
stopwords = nltk.corpus.stopwords.words("english")
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [25]:
df.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id'],
      dtype='object')

In [ ]:
# clean brackets
import re
def clean_text(text):
    # Remove brackets 
    cleaned_text = re.sub(r'[\(\)]', '', text)
    return cleaned_text
df['cleaned'] = df['title'].apply(lambda x: clean_text(x) if pd.notnull(x) else [])
df['cleaned'].head()

In [50]:
# vectorize
df['words'] = df['cleaned'].apply(lambda x: nltk.word_tokenize(str(x).lower()))
df['words'].head()

0    [clinical, features, of, culture-proven, mycop...
1    [nitric, oxide, :, a, pro-inflammatory, mediat...
2    [surfactant, protein-d, and, pulmonary, host, ...
3          [role, of, endothelin-1, in, lung, disease]
4    [gene, expression, in, epithelial, cells, in, ...
Name: words, dtype: object

In [ ]:
# lemmetization
nltk.download("wordnet")
nltk.download("omw-1.4")
wnl = nltk.WordNetLemmatizer()

In [51]:
# remove stopwords/punctuations, lemmatize
df['words'] = df['words'].apply(lambda word_list: [wnl.lemmatize(word) for word in word_list if word.isalpha() and word not in stopwords])
df['words'].head()

0    [clinical, feature, mycoplasma, pneumoniae, in...
1             [nitric, oxide, mediator, lung, disease]
2               [surfactant, pulmonary, host, defense]
3                                [role, lung, disease]
4    [gene, expression, epithelial, cell, response,...
Name: words, dtype: object

In [52]:
all_words = [word for sublist in df['words'] for word in sublist]
all_words[:5]

['clinical', 'feature', 'mycoplasma', 'pneumoniae', 'infection']

In [81]:
df_titles = pd.DataFrame(all_words)
df_titles.to_csv('titles.csv',index=False)

In [53]:
# frequency
fd = nltk.FreqDist(all_words)
fd.tabulate(10)

   pandemic     patient       study     disease      health   infection coronavirus      impact        case      review 
     140541      114811       80657       65229       63965       57523       54918       46870       44139       43879 


In [54]:
# bigram
finder2 = nltk.collocations.BigramCollocationFinder.from_words(all_words)
finder2.ngram_fd.tabulate(10)

 ('coronavirus', 'disease')    ('systematic', 'review')          ('case', 'report')        ('mental', 'health')    ('acute', 'respiratory')         ('cohort', 'study')        ('public', 'health')      ('impact', 'pandemic') ('respiratory', 'syndrome')          ('health', 'care') 
                      22357                       18180                       13575                       12965                       10732                        8468                        8365                        8238                        7893                        7584 


In [55]:
# trigram
finder3 = nltk.collocations.TrigramCollocationFinder.from_words(all_words)
finder3.ngram_fd.tabulate(10)

        ('severe', 'acute', 'respiratory')       ('acute', 'respiratory', 'syndrome') ('respiratory', 'syndrome', 'coronavirus')     ('coronavirus', 'disease', 'pandemic')           ('critically', 'ill', 'patient')              ('intensive', 'care', 'unit')      ('randomized', 'controlled', 'trial')    ('respiratory', 'distress', 'syndrome')       ('acute', 'respiratory', 'distress')       ('retrospective', 'cohort', 'study') 
                                      6931                                       6018                                       4867                                       3371                                       3340                                       3316                                       3157                                       2592                                       2543                                       2381 


In [ ]:
# repeat the process for abstracts

In [66]:
# clean brackets
df['cleaned_abs'] = df['abstract'].apply(lambda x: clean_text(x) if pd.notnull(x) else [])
df['cleaned_abs'].head()

0    OBJECTIVE: This retrospective chart review des...
1    Inflammatory diseases of the respiratory tract...
2    Surfactant protein-D SP-D participates in the ...
3    Endothelin-1 ET-1 is a 21 amino acid peptide w...
4    Respiratory syncytial virus RSV and pneumonia ...
Name: cleaned_abs, dtype: object

In [67]:
# vectorize
df['abs_words'] = df['cleaned_abs'].apply(lambda x: nltk.word_tokenize(str(x).lower()))
df['abs_words'].head()

0    [objective, :, this, retrospective, chart, rev...
1    [inflammatory, diseases, of, the, respiratory,...
2    [surfactant, protein-d, sp-d, participates, in...
3    [endothelin-1, et-1, is, a, 21, amino, acid, p...
4    [respiratory, syncytial, virus, rsv, and, pneu...
Name: abs_words, dtype: object

In [68]:
# remove stopwords/punctuations, lemmatize
df['abs_words'] = df['abs_words'].apply(lambda word_list: [wnl.lemmatize(word) for word in word_list if word.isalpha() and word not in stopwords])
df['abs_words'].head()

0    [objective, retrospective, chart, review, desc...
1    [inflammatory, disease, respiratory, tract, co...
2    [surfactant, participates, innate, response, i...
3    [amino, acid, peptide, diverse, biological, ac...
4    [respiratory, syncytial, virus, rsv, pneumonia...
Name: abs_words, dtype: object

In [69]:
all_abs_words = [word for sublist in df['abs_words'] for word in sublist]
all_abs_words[:5]

['objective', 'retrospective', 'chart', 'review', 'describes']

In [82]:
df_abstracts = pd.DataFrame(all_abs_words)
df_abstracts.to_csv('abstracts.csv',index=False)

In [70]:
# frequency
abs_fd = nltk.FreqDist(all_abs_words)
abs_fd.tabulate(10)

  patient     study  pandemic    result   disease    health infection      data    method      case 
  1123391    778194    544136    509470    502793    496989    402256    380355    371558    335178 


In [71]:
# bigram
finder2 = nltk.collocations.BigramCollocationFinder.from_words(all_abs_words)
finder2.ngram_fd.tabulate(10)

 ('coronavirus', 'disease')    ('acute', 'respiratory')        ('public', 'health')        ('mental', 'health') ('respiratory', 'syndrome')         ('severe', 'acute')          ('health', 'care') ('syndrome', 'coronavirus')          ('risk', 'factor')   ('infectious', 'disease') 
                     107952                       85359                       79401                       69907                       68801                       67107                       50516                       48703                       47493                       34581 


In [72]:
# trigram
finder3 = nltk.collocations.TrigramCollocationFinder.from_words(all_abs_words)
finder3.ngram_fd.tabulate(10)

        ('severe', 'acute', 'respiratory')       ('acute', 'respiratory', 'syndrome') ('respiratory', 'syndrome', 'coronavirus')              ('intensive', 'care', 'unit')     ('coronavirus', 'disease', 'pandemic')        ('world', 'health', 'organization') ('supplementary', 'material', 'available')  ('contains', 'supplementary', 'material')        ('polymerase', 'chain', 'reaction')    ('respiratory', 'distress', 'syndrome') 
                                     66176                                      61374                                      48335                                      24261                                      23086                                      15749                                      14207                                      13990                                      13892                                      12886 
